In [1]:
#To calculate loan score need three things
# If loan score is high ,higher the chnaces of loan getting approved
#1) laon repayment history (last payment , total payment received) 
#2) loan _defalters_history (delinq_2_yrs, pub_rec, pub_rec_bankruptcies, inq_last_6mths)
#3) financial health data (home ownership, loan status, grade pts)

In [2]:
#Now tables we have already created : will need help from these col names
# customers (home ownership, grade pts, high credit limit)
#loans (monthly installment , loan status , funded amount)
# loans_repay (last payment , total payment received)
#loans_delinq (delinq_2yrs)
#loans_enqq (pub_rec, pub_rec_bankruptcies,inq_last_6mths)

In [3]:
#Now Data is cleaned but then to there will be some bad data 
from pyspark.sql import SparkSession
import getpass
username=getpass.getuser()
spark=SparkSession\
    .builder\
    .config('spark.shuffle.useOldFetchProtocol','true')\
    .config('spark.ui.port','0')\
    .config("spark.sql.warehouse.dir","/user/itv016478/warehouse")\
    .enableHiveSupport()\
    .master('yarn')\
    .getOrCreate()

In [4]:
# 1) In customers table, bad data
spark.sql("select member_id , count(*) as total from itv016478_lending_club.customers group by member_id order by total desc")

member_id,total
e4c167053d5418230...,5
76b577467eda5bdbc...,4
3f87585a20f702838...,4
ad8e5d384dae17e06...,4
498bb6b1f0099cb47...,3
c92062bb371842b3d...,3
d3731bad3e6a82515...,3
3ae415acd6bbfaac1...,3
819453be77718d747...,3
4ab6205de571ccb7b...,3


In [5]:
# In customers table, you can see for one particular member_id there is  different credit score (total_high_credit_limit) 
#which should not be the case,there should be only one (total_high_credit_limit) for particular member_id
spark.sql("select * from itv005857_lending_club.customers where member_id like 'e4c167053d5418230%' ")

member_id,emp_title,emp_length,home_ownership,annual_income,address_state,address_zipcode,address_country,grade,sub_grade,verification_status,total_high_credit_limit,application_type,join_annual_income,verification_status_joint,ingest_date
e4c167053d5418230...,null,6,MORTGAGE,55000.0,IL,604xx,USA,B,B5,Verified,138780.0,Individual,null,null,2023-09-15 17:18:...
e4c167053d5418230...,null,6,MORTGAGE,55000.0,IL,604xx,USA,B,B5,Verified,171165.0,Individual,null,null,2023-09-15 17:18:...
e4c167053d5418230...,null,6,MORTGAGE,55000.0,IL,604xx,USA,B,B5,Verified,110907.0,Individual,null,null,2023-09-15 17:18:...
e4c167053d5418230...,null,6,MORTGAGE,55000.0,IL,604xx,USA,B,B5,Verified,129833.0,Individual,null,null,2023-09-15 17:18:...
e4c167053d5418230...,null,6,MORTGAGE,55000.0,IL,604xx,USA,B,B5,Verified,207300.0,Individual,null,null,2023-09-15 17:18:...


In [6]:
#Now some issues we will give to the upstream team
#problem a) For one member id there is 5 record..how will know which is the correct one .
#b) For one member id there is different total_high_credit_limit ..how will know which os correct one 

In [7]:
#2) For delinq table , bad data :
spark.sql("""select member_id, 
count(*) as total from itv005857_lending_club.loans_defaulters_delinq
group by member_id order by total desc""")

member_id,total
e4c167053d5418230...,3
55d55d97420671a1c...,2
58dde656f747cee8b...,2
c16e1f92ae5ccc785...,2
6c2b63ff231e520d4...,2
77db5fdf951dd04b2...,2
17d76ba5141e1c33a...,2
343344c3c65b023af...,2
bbe43331566910d55...,2
b5ded5638e54e1692...,2


In [8]:
#For (loans_defaulters_delinq) table also
#Here also for one particular member_id there should only be one row...so that i can trust the data ,but here its 3 rows for delinq 2 yrs
spark.sql("select * from itv005857_lending_club.loans_defaulters_delinq where member_id like 'e4c167053d5418230%' ")

member_id,delinq_2yrs,delinq_amnt,mths_since_last_delinq
e4c167053d5418230...,3,0.0,0
e4c167053d5418230...,1,0.0,0
e4c167053d5418230...,1,0.0,45


In [9]:
#3) For Enq table ,bad data ,
# for 1 member id there should be only 1 inquiry in last 6 mnths
spark.sql("select member_id, count(*) as total from itv005857_lending_club.loans_defaulters_detail_rec_enq group by member_id order by total desc")

member_id,total
e3b0c44298fc1c149...,33
e4c167053d5418230...,5
76b577467eda5bdbc...,4
ad8e5d384dae17e06...,4
3f87585a20f702838...,4
53789bea7edc660ed...,3
059d401bb603d9a80...,3
035bf3d8288d803bd...,3
f54295a60946dedad...,3
291ca1b09ef11ca3e...,3


In [10]:
# 4) Seeing member id for customers table : if want to see member id and total count  then write this - writing query where member id is coming more than 1 times
bad_data_customer_df = spark.sql("select member_id, count(*) as total from itv016478_lending_club.customers group by member_id having total > 1")
bad_data_customer_df.count()

3157

In [11]:
#4) Seeing member id for customers table : if you only want to see member id then -  writing query where member id is coming more than 2 times
bad_data_customer_df = spark.sql("select member_id from ( select member_id, count(*) as total from itv016478_lending_club.customers group by member_id having total > 1)")
bad_data_customer_df

member_id
7115ace193e13d8f3...
d4782ddad8591f44d...
eebbd89aa7efc13e7...
61be6498c93cadf89...
5130d0087970e032e...
cbede54df344cdb94...
2bae2e4dd6d5f2b21...
f284044b881f218c0...
a2af7506825a7dcff...
675151e58a628e87b...


In [12]:
#5) seeing membr id for delinq ,
bad_data_loans_defaulters_delinq_df= spark.sql("select member_id from ( select member_id, count(*) as total from itv016478_lending_clubb.loans_delinq group by member_id having total > 1)")
bad_data_loans_defaulters_delinq_df.count()

939

In [13]:
bad_data_loans_defaulters_enq_df= spark.sql("select member_id from ( select member_id, count(*) as total from itv016478_lending_clubb.loans_enq group by member_id having total > 1)")
bad_data_loans_defaulters_enq_df.count()

3189

In [14]:
bad_data_customer_df.repartition(1).write \
.option("header",True) \
.format("csv") \
.mode("overwrite") \
.option("path", "/user/itv016478/lendingclubproject/bad/bad_data_customers")\
.save()

In [15]:
bad_data_loans_defaulters_delinq_df.repartition(1).write \
.option("header",True) \
.format("csv") \
.mode("overwrite") \
.option("path", "/user/itv016478/lendingclubproject/bad/bad_data_delinq")\
.save()

In [16]:
bad_data_loans_defaulters_enq_df.repartition(1).write \
.option("header",True) \
.format("csv") \
.mode("overwrite") \
.option("path", "/user/itv016478/lendingclubproject/bad/bad_data_enq")\
.save()

In [17]:
# member ids are still repeting ie bad records (member _id)
#customers( df1 ) 3157
# delinq ( df2 )939
# enqq ( df3 )3189
# a consolidate file which has all unique  and not repeating member ids from above 3 dfs 
# Lets union / combine all  ( df1 ,df2 , df3 ) then do distinct (to find unique values) so that we will get all member_id 1 time

In [18]:
bad_record_data_df= bad_data_customer_df.select("member_id") \
.union(bad_data_loans_defaulters_delinq_df.select("member_id")) \
.union(bad_data_loans_defaulters_enq_df.select("member_id")) 

In [19]:
bad_data_final_df = bad_record_data_df.distinct()


In [20]:
bad_data_final_df.count()

3189

In [21]:
#Now member_id will not repeat . Hence storing the data in disk
bad_data_final_df.repartition(1).write \
.option("header",True) \
.format("csv") \
.mode("overwrite") \
.option("path", "/user/itv016478/lendingclubproject/bad/bad_data_final")\
.save()

In [22]:
#Segregating Indentified Bad data from Normal Data (Cleaned folder)
bad_data_final_df.createOrReplaceTempView("bad_data_final")

In [23]:
#1) segregating for customers 
customers_final_df = spark.sql("select * from itv016478_lending_clubb.customers where member_id NOT IN ( select member_id from bad_data_final)")


In [24]:
customers_final_df.count()

2254192

In [25]:
# storing new data for customers table  in new file "new _cleaned"-> Customers
customers_final_df.write\
.format("parquet")\
.mode("overwrite")\
.option("path","/user/itv016478/lendingclubproject/cleaned_new/customers_parquet")\
.save()

In [26]:
#2) segregating for delinq
delinq_final_df= spark.sql("select * from itv016478_lending_clubb.loans_delinq where member_id NOT IN ( select member_id from bad_data_final)")
delinq_final_df.count()

1103094

In [27]:
# storing new data for customers table  in new file "new _cleaned"-> delinq
delinq_final_df.write\
.format("parquet")\
.mode("overwrite")\
.option("path","/user/itv016478/lendingclubproject/cleaned_new/delinq_parquet")\
.save()

In [28]:
enqq_final_df= spark.sql("select * from itv016478_lending_clubb.loans_enqq where member_id NOT IN ( select member_id from bad_data_final)")
enqq_final_df.count()

2254195

In [29]:
# storing new data for customers table  in new file "new _cleaned"-> enqq
enqq_final_df.write\
.format("parquet")\
.mode("overwrite")\
.option("path","/user/itv016478/lendingclubproject/cleaned_new/enqq_parquet")\
.save()

In [30]:
#Now creating a external table based on this new location 


In [31]:
#1)for customers
spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS itv016478_lending_clubb.new_customers(
member_id string, emp_title string, emp_length int , home_ownership string,
annual_inc float, addr_state string, zip_code string, country string, grade string,
sub_grade string, verification_status string, tot_hi_cred_lim float, application_type string,
annual_inc_joint float, verification_status_joint string, ingest_date timestamp )
stored as parquet LOCATION' /user/itv016478/lendingclubproject/cleaned_new/customers_parquet' """)


""


In [32]:
#2) for delinq
spark.sql("""CREATE EXTERNAL TABLE IF NOT EXISTS itv016478_lending_clubb.loans_new_delinq(
member_id string, delinq_2yrs integer, delinq_amnt float, mths_since_last_delinq FLOAT)
stored as parquet LOCATION '/user/itv016478/lendingclubproject/cleaned_new/delinq_parquet'""")

""


In [33]:
#3)for enqq -
spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS itv016478_lending_clubb.loans_new_enqq(
    member_id STRING,
    pub_rec FLOAT,
    pub_rec_bankruptcies FLOAT,
    inq_last_6mths FLOAT
)
STORED AS PARQUET
LOCATION '/user/itv016478/lendingclubproject/cleaned_new/enqq_parquet'
""")

""


In [46]:
#TABLE CREATED BY TRENDYTECH TEAM [YOU CAN USE THIS BUT HERE WE HAVE GIVEN THE LOCATION OF PUBLIC DIRECTORY YOU CAN'T MAKE ANY CHANGES IN THAT DIRECTORY] 
spark.sql("""
create EXTERNAL TABLE itv016478_lending_clubb.loans_new_enqq_tt(member_id string, pub_rec integer, pub_rec_bankruptcies integer, inq_last_6mths integer)
stored as parquet
LOCATION '/public/trendytech/lendingclubproject/cleaned_new/loan_defaulters_detail_rec_enq_parquet'
""")

""


In [47]:
spark.sql("DESCRIBE FORMATTED itv016478_lending_clubb.loans_new_enqq_tt").show(truncate=False)

+----------------------------+---------------------------------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                                            |comment|
+----------------------------+---------------------------------------------------------------------------------------------------------------------+-------+
|member_id                   |string                                                                                                               |null   |
|pub_rec                     |int                                                                                                                  |null   |
|pub_rec_bankruptcies        |int                                                                                                                  |null   |
|inq_last_6mths              |int                         

In [48]:
spark.sql("SELECT * FROM itv016478_lending_clubb.loans_new_enqq_tt").show()

+--------------------+-------+--------------------+--------------+
|           member_id|pub_rec|pub_rec_bankruptcies|inq_last_6mths|
+--------------------+-------+--------------------+--------------+
|9cb79aa7323e81be1...|      0|                   0|             0|
|0dd2bbc517e3c8f9e...|      1|                   1|             3|
|458458599d3df3bfc...|      1|                   1|             1|
|05ea141ec28b5c7f7...|      0|                   0|             0|
|aac68850fdac09fd0...|      0|                   0|             0|
|3a423e4589e89f429...|      0|                   0|             0|
|f1efcf7dfbfef21be...|      0|                   0|             1|
|c89986155a070db2e...|      0|                   0|             1|
|118dc629b6e134419...|      0|                   0|             0|
|a86fa4b7493708333...|      0|                   0|             0|
|6e8d94bf446e97025...|      0|                   0|             0|
|3de585156dc6b73f6...|      0|                   0|           

In [37]:
spark.sql("DESCRIBE FORMATTED itv016478_lending_clubb.loans_new_enqq").show(truncate=False)

+----------------------------+----------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                               |comment|
+----------------------------+----------------------------------------------------------------------------------------+-------+
|member_id                   |string                                                                                  |null   |
|pub_rec                     |int                                                                                     |null   |
|pub_rec_bankruptcies        |int                                                                                     |null   |
|inq_last_6mths              |int                                                                                     |null   |
|                            |                                                                          

In [38]:
#Now check the member_id is no
t repating for customers table
spark.sql("select member_id , count(*) as total from itv016478_lending_clubb.new_customers group by member_id order by total desc")

SyntaxError: invalid syntax (<ipython-input-38-870eedb0c929>, line 2)

In [ ]:
#Now check the member_id is not repating for delinq table
spark.sql("select member_id , count(*) as total from itv016478_lending_clubb.loans_new_delinq group by member_id order by total desc")

In [ ]:
#Now check the member_id is not repating for enqq table
spark.sql("select member_id , count(*) as total from itv016478_lending_clubb.loans_new_enqq group by member_id order by total desc")

In [ ]:
#So now we have segregated bad data (repeating member id to upstream team to check why are we geeting bad data after cleaning )
# We have cleaned one more layer and hence we have unique member_id now and now we can proceed processing  with loan score
#Now there is no issues with data.